# CSV Builder

## Importo librerias necesarias

In [1]:
#Pandas
import pandas as pd
#Numpy
import numpy as np
#Datetime
import datetime as dt
#Pendulum
import pendulum

## Files necesarios

In [2]:
# Nota: Colocarlos en la carpeta src.
# valor_educacion.csv
# fiuba_1_postulantes_educacion.csv
# fiuba_1_postulantes_educacion_2.csv
# fiuba_1_postulantes_educacion_3.csv
# fiuba_2_postulantes_genero_y_edad.csv
# fiuba_2_postulantes_genero_y_edad_2.csv
# fiuba_2_postulantes_genero_y_edad_3.csv
# fiuba_3_vistas.csv
# fiuba_3_vistas_2.csv
# fiuba_3_vistas_3.csv
# fiuba_4_postulaciones.csv
# fiuba_4_postulaciones_2.csv
# fiuba_5_avisos_online.csv
# fiuba_5_avisos_online_2.csv
# fiuba_6_avisos_detalle.csv
# fiuba_6_avisos_detalle_2.csv
# fiuba_6_avisos_detalle_3.csv
# fiuba_6_avisos_detalle_4.csv

## Proceso a los postulantes

## Proceso informacion academica de los postulantes

In [3]:
#Informacion de nivel educativo de los postulantes
post_educacion = pd.concat([
    pd.read_csv('src/fiuba_1_postulantes_educacion.csv'), 
    pd.read_csv('src/fiuba_1_postulantes_educacion_2.csv'),
    pd.read_csv('src/fiuba_1_postulantes_educacion_3.csv')], axis=0)

In [4]:
#Mapeo de puntaje del nivel educativo
post_educacion = pd.merge(
    post_educacion,
    pd.read_csv('src/valor_educacion.csv', encoding='utf-8'),
    how='inner', left_on=['nombre','estado'], right_on = ['nombre','estado'])

In [5]:
# Busco la maxima educacion que tiene cada postulante.
max_post_educacion = post_educacion.groupby(['idpostulante'])['valor_educacion'].max().reset_index()
# Hago el join para filtrar todas las educaciones que no me interesan, dejando solo la maxima educacion.
post_educacion = pd.merge(post_educacion, max_post_educacion,  how='inner',left_on=['idpostulante','valor_educacion'],right_on = ['idpostulante','valor_educacion'])
post_educacion.drop(['nombre', 'estado'], inplace = True, axis=1)
post_educacion.drop_duplicates(inplace=True)

## Proceso informacion de genero y edad de los postulantes

In [6]:
#Informacion de genero y edad de los postulantes
post_gen_edad = pd.concat([
    pd.read_csv('src/fiuba_2_postulantes_genero_y_edad.csv'),
    pd.read_csv('src/fiuba_2_postulantes_genero_y_edad_2.csv'),
    pd.read_csv('src/fiuba_2_postulantes_genero_y_edad_3.csv')], axis=0)

In [7]:
post_gen_edad['sexo'] = post_gen_edad['sexo'].astype('category')

In [8]:
post_gen_edad.drop_duplicates(inplace=True)

In [9]:
post_gen_edad.loc[post_gen_edad['sexo'] == '0.0','sexo'] = np.nan
post_gen_edad.loc[post_gen_edad['sexo'] == 'NO_DECLARA','sexo'] = np.nan

cols = post_gen_edad.select_dtypes(['category']).columns
post_gen_edad[cols] = post_gen_edad[cols].apply(lambda x: x.cat.codes)

In [10]:
# Modifico el tipo de dato de la fecha.
post_gen_edad.fechanacimiento = post_gen_edad.fechanacimiento.apply(pd.to_datetime, errors='coerce')

now = pd.Timestamp(dt.datetime.now())
pd.to_datetime(now)
post_gen_edad['edad'] = (now - pd.to_datetime(post_gen_edad['fechanacimiento'])).astype('<m8[Y]')
post_gen_edad.drop('fechanacimiento', inplace=True, axis=1)

In [11]:
post_gen_edad.drop_duplicates('idpostulante', inplace=True) 

In [12]:
post_gen_edad.fillna(0, inplace=True)

In [13]:
post_gen_edad['edad'] = post_gen_edad['edad'].astype(int)

In [14]:
post_gen_edad['sexo'] = post_gen_edad['sexo'].apply(lambda x: 0 if x == -1 else (-1 if x == 2 else 1))

In [15]:
#Mergeo la nueva informacion
postulantes = pd.merge(post_educacion, post_gen_edad, how='outer', on='idpostulante')

In [16]:
postulantes.fillna(-1, inplace=True)

## Proceso a las vistas

In [17]:
vistas = pd.concat([
    pd.read_csv('src/fiuba_3_vistas.csv'),
    pd.read_csv('src/fiuba_3_vistas_2.csv'),
    pd.read_csv('src/fiuba_3_vistas_3.csv')], axis=0)

In [18]:
vistas = vistas.drop_duplicates()
vistas = vistas.rename({'idAviso': 'idaviso'}, axis=1)

In [19]:
vistas['timestamp'] = vistas['timestamp'].apply(lambda x: pendulum.parse(x))

In [20]:
vistas['month']     = vistas['timestamp'].apply(lambda x: x.month).astype(int) 
vistas['day']       = vistas['timestamp'].apply(lambda x: x.day).astype(int) 
vistas['dayofweek'] = vistas['timestamp'].apply(lambda x: x.day_of_week).astype(int) 
vistas['hour']      = vistas['timestamp'].apply(lambda x: x.hour).astype(int) 

In [21]:
vistas = vistas.drop('timestamp', axis=1)

In [22]:
vistas = vistas.fillna(-1)

## Proceso mas postulaciones

In [23]:
postulaciones = pd.concat([
    pd.read_csv('src/fiuba_4_postulaciones.csv'),
    pd.read_csv('src/fiuba_4_postulaciones_2.csv')], axis=0)

In [24]:
postulaciones.drop_duplicates(inplace=True)

In [25]:
postulaciones.rename({'fechapostulacion': 'timestamp'}, axis=1, inplace=True)

In [26]:
postulaciones['timestamp'] = postulaciones['timestamp'].apply(lambda x: pendulum.parse(x))

In [27]:
postulaciones['month']     = postulaciones['timestamp'].apply(lambda x: x.month).astype(int) 
postulaciones['day']       = postulaciones['timestamp'].apply(lambda x: x.day).astype(int) 
postulaciones['dayofweek'] = postulaciones['timestamp'].apply(lambda x: x.dayofweek).astype(int) 
postulaciones['hour']      = postulaciones['timestamp'].apply(lambda x: x.hour).astype(int) 

In [28]:
postulaciones.drop('timestamp', axis=1, inplace=True)

## Proceso Avisos Online

In [29]:
avisos_online = pd.concat([
    pd.read_csv('src/fiuba_5_avisos_online.csv'),
    pd.read_csv('src/fiuba_5_avisos_online_2.csv')], axis=0)

In [30]:
avisos_online.drop_duplicates(inplace=True)

## Proceso el detalle de los avisos

In [31]:
avisos_detalles = pd.concat([
    pd.read_csv('src/fiuba_6_avisos_detalle.csv'),
    pd.read_csv('src/fiuba_6_avisos_detalle_2.csv'),
    pd.read_csv('src/fiuba_6_avisos_detalle_3.csv'),
    pd.read_csv('src/fiuba_6_avisos_detalle_4.csv')], axis=0)

In [32]:
avisos_detalles.drop('idpais', axis=1, inplace=True)
avisos_detalles.drop_duplicates('idaviso', inplace=True)

In [33]:
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Bioquímica','nombre_area'] = 'Bioquimica'
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Química','nombre_area'] = 'Quimica'
avisos_detalles.loc[avisos_detalles['nombre_area'] == 'Trabajo social','nombre_area'] = 'Trabajo Social'

## Detalle Avisos (Nombre Zona)

In [34]:
avisos_detalles_zona = pd.DataFrame(avisos_detalles['nombre_zona'].unique(), columns=['nombre_zona'])
avisos_detalles_zona.index.name = 'idzona'
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_zona, how='inner', on='nombre_zona')
avisos_detalles.drop('nombre_zona', inplace=True, axis=1)

## Detalle Avisos (Nombre Ciudad)

In [35]:
avisos_detalles_ciudad = pd.DataFrame(avisos_detalles['ciudad'].unique(), columns=['ciudad'])
avisos_detalles_ciudad.index.name = 'idciudad'
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_ciudad, how='inner', on='ciudad')
avisos_detalles.drop('ciudad', inplace=True, axis=1)

## Detalle Avisos (Mapa Calle)

In [36]:
avisos_detalles_mapacalle = pd.DataFrame(avisos_detalles['mapacalle'].unique(), columns=['mapacalle'])
avisos_detalles_mapacalle.index.name = 'idmapacalle'
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_mapacalle, how='inner', on='mapacalle')
avisos_detalles.drop('mapacalle', inplace=True, axis=1)

## Detalle Avisos (Tipo de Trabajo)

In [37]:
avisos_detalles_tipo_de_trabajo = pd.DataFrame(avisos_detalles['tipo_de_trabajo'].unique(), columns=['tipo_de_trabajo'])
avisos_detalles_tipo_de_trabajo.index.name = 'idtipo_de_trabajo'
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_tipo_de_trabajo, how='inner', on='tipo_de_trabajo')
avisos_detalles.drop('tipo_de_trabajo', inplace=True, axis=1)

## Detalle Avisos (Nivel Laboral)

In [38]:
avisos_detalles_nivel_laboral = pd.DataFrame(avisos_detalles['nivel_laboral'].unique(), columns=['nivel_laboral'])
avisos_detalles_nivel_laboral.index.name = 'idnivel_laboral'
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_nivel_laboral, how='inner', on='nivel_laboral')
avisos_detalles.drop('nivel_laboral', inplace=True, axis=1)

## Detalle Avisos (Nombre Area)

In [39]:
avisos_detalles_nombre_area = pd.DataFrame(avisos_detalles['nombre_area'].unique(), columns=['nombre_area'])
avisos_detalles_nombre_area.sort_values('nombre_area', inplace=True)
avisos_detalles_nombre_area.reset_index(inplace=True)
avisos_detalles_nombre_area.drop('index', axis=1, inplace=True)
avisos_detalles_nombre_area.index.name = 'idnombre_area'
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_nombre_area, how='inner', on='nombre_area')
avisos_detalles.drop('nombre_area', inplace=True, axis=1)

## Detalle Avisos (Denominacion Empresa)

In [40]:
#avisos_detalles_denominacion_empresa = pd.DataFrame(avisos_detalles['denominacion_empresa'].unique(), columns=['denominacion_empresa'])
#avisos_detalles_denominacion_empresa.index.name = 'iddenominacion_empresa'
#avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_denominacion_empresa, how='inner', on='denominacion_empresa')
#avisos_detalles.drop('denominacion_empresa', inplace=True, axis=1)

## Agrego informacion de la publicacion y del postulante

In [41]:
#No realizo el merge de titulo y descripcion del aviso. Asi que no tendre esta informacion del aviso en le futuro.
#pd.merge(avisos_detalles, postulaciones, how='inner', on='idaviso')

In [42]:
#Agrego la informacion del postulante a dicha postulacion
postulaciones = pd.merge(postulaciones, postulantes, how='inner', on='idpostulante')

## Columna "se_postulo"

### Si es una vista todabia no se postulo...

In [43]:
vistas['se_postulo'] = 0

### Si es una postulacion ya se postulo...

In [44]:
postulaciones['se_postulo'] = 1

## Hago el concat de las Vistas con las Postulaciones

In [45]:
vistas_postulaciones = pd.concat([postulaciones, vistas], axis=0)

In [46]:
vistas_postulaciones = vistas_postulaciones.fillna(-1)

In [47]:
vistas_postulaciones['valor_educacion'] = vistas_postulaciones['valor_educacion'].astype(int)

In [48]:
vistas_postulaciones.head()

,day,dayofweek,edad,hour,idaviso,idpostulante,month,se_postulo,sexo,valor_educacion
0,19,4,37.0,7,1112248724,NjlD,1,1,1.0,12
1,24,2,39.0,15,1112286523,ZaO5,1,1,1.0,11
2,24,2,39.0,15,1112272060,ZaO5,1,1,1.0,11
3,26,4,39.0,8,1112288401,ZaO5,1,1,1.0,11
4,30,1,39.0,13,1112300563,ZaO5,1,1,1.0,11


## Exporto finalmente el .CSV

In [49]:
vistas_postulaciones.to_csv('src.csv', encoding='utf-8', index=False)